In [5]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.1 MB/s eta 0:00:00


# Documentacion


## # Proyecto de Inteligencia de Enjambre - VRP con ACO

Este proyecto implementa un algoritmo basado en colonias de hormigas para resolver problemas de ruteo de vehículos con capacidad limitada y demandas de clientes. La idea es generar rutas eficientes minimizando la distancia total y respetando las restricciones de capacidad.

## Documentación de las fuentes

La parte central del código está en la clase `ACOSolver`. Esta clase recibe parámetros como `alpha` y `beta` que controlan cuánto peso tienen las feromonas y la heurística en la selección de nodos, `rho` que es la tasa de evaporación, el número de iteraciones que se harán y cuántas hormigas participan en la construcción de soluciones. La función principal es `solve`, que recibe los nodos, la matriz de distancias y la capacidad del vehículo, y devuelve las rutas optimizadas. Dentro de `solve`, primero se preparan los datos, luego se ejecuta el ACO y finalmente se hace una optimización local 2-opt para mejorar cada ruta.

Hay funciones auxiliares que ayudan en el preprocesamiento y construcción de soluciones. Por ejemplo, `greedy_cluster_mejorado` agrupa los clientes en clusters respetando la capacidad del vehículo y priorizando clientes de mayor demanda y cercanía, mientras que `aplicar_aco_solver_clusters` aplica el solver a cada cluster y calcula la distancia total de todas las rutas. La clase `Graph` simplemente facilita la carga de datos desde archivos parquet, entregando tanto los nodos como la matriz de distancias de forma práctica.

El proyecto también incluye una función `evaluar_algoritmo_aco`, que recorre un conjunto de problemas, genera clusters, aplica el solver y compara los resultados con un benchmark, calculando errores, tiempo de ejecución y memoria utilizada.

En cuanto a los parámetros, las entradas principales de la función `solve` son los nodos (con coordenadas y demandas), la matriz de distancias y la capacidad. Los hiperparámetros son `alpha`, `beta`, `rho`, número de iteraciones y número de hormigas. La salida son las rutas encontradas y la distancia total, que se puede obtener con `solution_length`.

## Análisis comparativo de estadísticas

Para evaluar cómo funciona nuestro ACO frente al benchmark, se calculan varios errores y métricas:

- Error absoluto: diferencia entre la distancia calculada por ACO y la distancia del benchmark.
- Error relativo: relación entre el error absoluto y la distancia del benchmark.
- Error promedio (MAE) y error cuadrático promedio (MSE) a lo largo de todos los problemas.
- Además se mide el tiempo de ejecución y la memoria utilizada.

Estas métricas permiten ver si el algoritmo se acerca a la solución óptima y cómo se comporta en términos de eficiencia y uso de recursos. También se pueden generar gráficos de comparación entre ACO y benchmark para visualizar tendencias y ver cuáles problemas presentan mayores diferencias.

## Diagrama de flujo de datos

El flujo de datos en el proyecto es sencillo: primero se cargan los nodos y la matriz de distancias, se obtiene la información de coordenadas y demandas y se define la capacidad de los vehículos. Luego se realiza el clustering greedy para agrupar clientes por vehículo. Cada cluster pasa al ACO, que construye rutas y las optimiza localmente. Finalmente, se calculan las distancias, se comparan con el benchmark, se calculan los errores y se exportan los resultados.

```text
Archivos Benchmark / parquet
        │
        ▼
Cargar nodos y matriz de distancias
        │
        ▼
Preprocesamiento (coords, demandas, capacidad)
        │
        ▼
Clustering greedy por capacidad
        │
        ▼
Aplicar ACO a cada cluster
        │
        ▼
Optimización local 2-opt
        │
        ▼
Rutas finales y distancia total
        │
        ▼
Comparación con benchmark y cálculo de errores
        │
        ▼
Exportación de resultados a Excel


In [6]:
%pip install openpyxl

In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
from collections import Counter
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [8]:

import streamlit as st
import matplotlib.pyplot as plt
import altair as alt

# Clases

## ACO

In [10]:
import numpy as np
import pandas as pd


class ACOSolver:
    def __init__(self, alpha=1.0, beta=2.0, rho=0.1, max_iters=50, ants=20, distance_matrix=None, demands=None, depot_distances=None):
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.max_iters = max_iters
        self.ants = ants
        self.distance_matrix = distance_matrix
        self.demands = demands
        self.depot_distances = depot_distances
        self.n_nodes = len(demands) if demands is not None else 0

    def sum_length(self, route):
        if not route: return 0
        dist = self.depot_distances[route[0]]
        for i in range(len(route)-1):
            dist += self.distance_matrix[route[i]][route[i+1]]
        dist += self.depot_distances[route[-1]]
        return dist
        total_distance = 0
        # Depot -> primer nodo
        total_distance += self.depot_distances[route[0]]

        # Distancias entre nodos de la ruta
        for i in range(len(route) - 1):
            total_distance += self.distance_matrix[route[i]][route[i + 1]]

        # Último nodo -> depot
        total_distance += self.depot_distances[route[-1]]

        return total_distance

    def solution_length(self, routes):
        return sum(self.sum_length(r) for r in routes)

    def solve(self, nodes, distance_matrix, cap):
        """Método principal de resolución"""
        try:
            # Preparar datos
            self._prepare_data(nodes, distance_matrix)

            # Debug: verificar datos
            print(f"Debug: n_nodes={self.n_nodes}, capacity={cap}")
            print(f"Debug: demands shape={len(self.demands)}, depot_distances shape={len(self.depot_distances)}")

            # Resolver con ACO capacitado
            raw_paths = self._capacitated_aco(cap)
            print(f"Debug: raw_paths encontrados: {len(raw_paths) if raw_paths else 0}")

            if not raw_paths:
                print("Warning: No se encontraron rutas en ACO capacitado")
                return []

            # Optimizar cada ruta localmente
            optimized_paths = []
            for i, path in enumerate(raw_paths):
                if path and len(path) > 0:
                    optimized_path = self._local_search_2opt(path)
                    if optimized_path and len(optimized_path) > 0:
                        optimized_paths.append(optimized_path)
                        print(f"Debug: ruta {i} optimizada: {optimized_path}")

            print(f"Debug: {len(optimized_paths)} rutas finales")
            return optimized_paths

        except Exception as e:
            print(f"Error en solve: {str(e)}")
            import traceback
            traceback.print_exc()
            return []

    def _prepare_data(self, nodes, distance_matrix):
        """Prepara los datos para el algoritmo"""
        # Agregar distancias al depot
        nodes = nodes.copy()
        nodes["d0"] = distance_matrix.iloc[0]

        # Extraer datos necesarios
        self.depot_distances = nodes["d0"].values
        self.demands = nodes["demand"].values
        self.distance_matrix = distance_matrix.values
        self.n_nodes = len(nodes)

        print(f"Debug: Datos preparados - nodos: {self.n_nodes}")

    def _capacitated_aco(self, capacity):
        """ACO principal con restricciones de capacidad"""
        try:
            # Preparar matrices
            d = self.distance_matrix.copy()
            np.fill_diagonal(d, np.inf)

            tau = np.ones_like(d) * 0.1  # Feromonas iniciales pequeñas
            eta = np.divide(1.0, d, out=np.zeros_like(d), where=d>0)
            eta[eta == np.inf] = 0

            best_routes = []
            best_distance = float('inf')

            for iteration in range(self.max_iters):
                iteration_routes = []

                for ant in range(self.ants):
                    routes = self._construct_solution(tau, eta, capacity)
                    if routes:
                        iteration_routes.extend(routes)
                        total_distance = self.solution_length(routes)

                        if total_distance < best_distance:
                            best_distance = total_distance
                            best_routes = [route.copy() for route in routes]

                # Actualizar feromonas
                if iteration_routes:
                    self._update_pheromones(tau, iteration_routes)

                if iteration % 10 == 0:
                    print(f"Debug: Iteración {iteration}, mejor distancia: {best_distance:.2f}")

            print(f"Debug: ACO terminado, {len(best_routes)} rutas encontradas")
            return best_routes

        except Exception as e:
            print(f"Error en _capacitated_aco: {str(e)}")
            return []

    def _construct_solution(self, tau, eta, capacity):
        """Construye una solución completa respetando capacidades"""
        try:
            unvisited = set(range(1, self.n_nodes))  # Excluir depot (nodo 0)
            routes = []

            while unvisited:
                route = self._construct_single_route(tau, eta, capacity, unvisited)
                if route:
                    routes.append(route)
                    # Remover nodos visitados
                    for node in route:
                        unvisited.discard(node)
                else:
                    # Si no se puede construir más rutas, forzar asignación de nodos restantes
                    if unvisited:
                        remaining = list(unvisited)
                        # Crear rutas individuales para nodos que no caben
                        for node in remaining:
                            if self.demands[node] <= capacity:
                                routes.append([node])
                                unvisited.remove(node)
                    break

            return routes

        except Exception as e:
            print(f"Error en _construct_solution: {str(e)}")
            return []

    def _construct_single_route(self, tau, eta, capacity, available_nodes):
        """Construye una sola ruta respetando capacidad"""
        try:
            route = []
            current_capacity = capacity
            remaining_nodes = available_nodes.copy()
            current_node = 0  # Empezar desde depot

            while remaining_nodes:
                # Filtrar nodos que caben en la capacidad restante
                feasible_nodes = [node for node in remaining_nodes
                                if self.demands[node] <= current_capacity]

                if not feasible_nodes:
                    break

                # Calcular probabilidades de selección
                probabilities = []
                total_weight = 0

                for node in feasible_nodes:
                    pheromone = tau[current_node, node] if current_node < len(tau) else 0.1
                    heuristic = eta[current_node, node] if current_node < len(eta) else 1.0
                    weight = (pheromone ** self.alpha) * (heuristic ** self.beta)
                    probabilities.append(weight)
                    total_weight += weight

                if total_weight == 0:
                    # Selección aleatoria si no hay información
                    next_node = np.random.choice(feasible_nodes)
                else:
                    # Selección probabilística
                    probabilities = [p / total_weight for p in probabilities]
                    next_node = np.random.choice(feasible_nodes, p=probabilities)

                # Agregar nodo a la ruta
                route.append(next_node)
                remaining_nodes.remove(next_node)
                current_capacity -= self.demands[next_node]
                current_node = next_node

            return route if route else []

        except Exception as e:
            print(f"Error en _construct_single_route: {str(e)}")
            return []

    def _update_pheromones(self, tau, routes):
        """Actualiza feromonas"""
        try:
            # Evaporación
            tau *= (1 - self.rho)

            # Depositar feromonas
            for route in routes:
                if not route or len(route) == 0:
                    continue

                route_distance = self.sum_length(route)
                if route_distance > 0:
                    pheromone_deposit = 1.0 / route_distance

                    # Depot -> primer nodo
                    if route[0] < len(tau):
                        tau[0, route[0]] += pheromone_deposit
                        tau[route[0], 0] += pheromone_deposit

                    # Nodos consecutivos en la ruta
                    for i in range(len(route) - 1):
                        if route[i] < len(tau) and route[i+1] < len(tau):
                            tau[route[i], route[i+1]] += pheromone_deposit
                            tau[route[i+1], route[i]] += pheromone_deposit

                    # Último nodo -> depot
                    if route[-1] < len(tau):
                        tau[route[-1], 0] += pheromone_deposit
                        tau[0, route[-1]] += pheromone_deposit

        except Exception as e:
            print(f"Error en _update_pheromones: {str(e)}")

    def _local_search_2opt(self, route):
        """Optimización local 2-opt simple"""
        if not route or len(route) <= 2:
            return route

        try:
            best_route = route.copy()
            best_distance = self.sum_length(best_route)
            improved = True

            while improved:
                improved = False
                for i in range(1, len(route) - 1):
                    for j in range(i + 1, len(route)):
                        # Crear nueva ruta con 2-opt swap
                        new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
                        new_distance = self.sum_length(new_route)

                        if new_distance < best_distance:
                            best_route = new_route
                            best_distance = new_distance
                            improved = True
                            break
                    if improved:
                        break
                route = best_route

            return best_route
        except:
            return route


def process_single_instance(args):
    """Procesa una instancia individual del dataset"""
    idx, row = args

    try:
        print(f"Procesando instancia {idx}")

        # Cargar datos
        nodes = pd.read_parquet(row['nodes'])
        distance_matrix = pd.read_parquet(row['distance_matrix'])
        vehicle_capacity = row['vehicle_capacity']
        problem_cluster = row['problem_cluster']

        print(f"Instancia {idx}: {len(nodes)} nodos, capacidad {vehicle_capacity}")

        # Configurar solver
        aco_solver = ACOSolver(
            alpha=1.0,
            beta=2.0,
            rho=0.1,
            max_iters=30,  # Reducido para debugging
            ants=min(15, len(nodes))
        )

        # Resolver
        best_routes = aco_solver.solve(nodes, distance_matrix, vehicle_capacity)

        if best_routes and len(best_routes) > 0:
            best_value = aco_solver.solution_length(best_routes)
            n_vehicles = len(best_routes)
            print(f"Instancia {idx} resuelta: {n_vehicles} vehículos, distancia {best_value:.2f}")
        else:
            print(f"Instancia {idx} falló: no se encontraron rutas")
            best_value = None
            n_vehicles = 0

        return {
            'problem_cluster': problem_cluster,
            'vehicle_capacity': vehicle_capacity,
            'aco_best_route': best_routes,
            'aco_best_value': best_value,
            'aco_n_vehicles': n_vehicles,
            'status': 'success' if best_routes else 'no_solution'
        }

    except Exception as e:
        print(f"Error procesando instancia {idx}: {str(e)}")
        import traceback
        traceback.print_exc()

        return {
            'problem_cluster': row.get('problem_cluster', 'unknown'),
            'vehicle_capacity': row.get('vehicle_capacity', 0),
            'aco_best_route': [],
            'aco_best_value': None,
            'aco_n_vehicles': 0,
            'status': f'error: {str(e)}'
        }


def process_dataset(df, n_workers=None):
    """Función principal para procesar el dataset"""
    if n_workers is None:
        n_workers = min(mp.cpu_count(), 4)  # Limitar workers para debugging

    print(f"Procesando {len(df)} instancias con {n_workers} workers")

    results = []

    if n_workers == 1:
        # Procesamiento secuencial para debugging
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            result = process_single_instance((idx, row))
            results.append(result)
    else:
        # Procesamiento paralelo
        with ProcessPoolExecutor(max_workers=n_workers) as executor:
            # Enviar trabajos
            future_to_idx = {
                executor.submit(process_single_instance, (idx, row)): idx
                for idx, row in df.iterrows()
            }

            # Recopilar resultados
            for future in tqdm(as_completed(future_to_idx),
                              total=len(future_to_idx),
                              desc="Procesando"):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    idx = future_to_idx[future]
                    print(f"Error en worker para índice {idx}: {e}")
                    results.append({
                        'problem_cluster': 'error',
                        'vehicle_capacity': 0,
                        'aco_best_route': [],
                        'aco_best_value': None,
                        'aco_n_vehicles': 0,
                        'status': f'worker_error: {str(e)}'
                    })

    return pd.DataFrame(results)


# Función de prueba simple
def test_single_instance():
    """Prueba con una instancia simple para verificar funcionamiento"""
    # Crear datos de prueba simples
    n_nodes = 6
    np.random.seed(42)

    # Crear nodos con demandas
    nodes = pd.DataFrame({
        'demand': [0, 10, 15, 20, 12, 8]  # depot tiene demanda 0
    })

    # Crear matriz de distancias simétrica
    coords = np.random.rand(n_nodes, 2) * 100
    dist_matrix = np.zeros((n_nodes, n_nodes))
    for i in range(n_nodes):
        for j in range(n_nodes):
            dist_matrix[i, j] = np.sqrt(np.sum((coords[i] - coords[j])**2))

    distance_matrix = pd.DataFrame(dist_matrix)
    capacity = 30

    print("Probando instancia simple...")
    solver = ACOSolver(max_iters=20, ants=10)
    routes = solver.solve(nodes, distance_matrix, capacity)

    print(f"Rutas encontradas: {routes}")
    if routes:
        print(f"Distancia total: {solver.solution_length(routes):.2f}")
        print(f"Número de vehículos: {len(routes)}")

    return routes

# Clusterizacion + ACO

In [11]:
def greedy_cluster_mejorado(coords, demandas, capacidad, matriz):
    """
    Clusterización greedy: respeta capacidad y selecciona clientes por demanda/distancia.
    """
    restantes = set(range(1, len(demandas)))
    clusters = []

    while restantes:
        cluster = [0]  # siempre empieza en el depósito
        carga, actual = 0, 0

        while True:
            candidatos = [j for j in restantes if demandas[j] + carga <= capacidad]
            if not candidatos:
                break

            siguiente = max(
                candidatos,
                key=lambda j: demandas[j] / (matriz[actual, j] + 1e-6)
            )

            cluster.append(siguiente)
            carga += demandas[siguiente]
            restantes.remove(siguiente)
            actual = siguiente

        clusters.append(cluster)
    return clusters


In [12]:
def aplicar_aco_solver_clusters(clusters, nodes, distance_matrix, capacity, ants, n_iter, alpha, beta, rho):
    resultados = []
    distancia_total = 0

    for cluster in clusters:
        # Extraemos subgrafo del cluster
        sub_nodes = nodes.iloc[cluster].reset_index(drop=True)
        sub_dm = distance_matrix.iloc[cluster, cluster].reset_index(drop=True).T.reset_index(drop=True)

        solver = ACOSolver(alpha=alpha, beta=beta, rho=rho, ants=ants, max_iters=n_iter)
        rutas = solver.solve(sub_nodes, sub_dm, capacity)

        if rutas:
            rutas_globales = [[cluster[i] for i in ruta] for ruta in rutas]
            resultados.extend(rutas_globales)
            distancia_total += solver.solution_length(rutas)

    return resultados, len(resultados), distancia_total


In [13]:
import os
import gdown
import zipfile
if not os.path.exists("carpeta_descomprimida"):

  file_path = "https://drive.google.com/uc?id=1WW7AnTSgSVX7Ktn2z6GyHkQN3nbmqaRa"
  output = "archive.zip"
  gdown.download(file_path, output, quiet=False)


  with zipfile.ZipFile(output, 'r') as zip_ref:
      zip_ref.extractall('carpeta_descomprimida')

  os.remove(output)

Downloading...
From: https://drive.google.com/uc?id=1WW7AnTSgSVX7Ktn2z6GyHkQN3nbmqaRa
To: /content/archive.zip
100%|██████████| 847k/847k [00:00<00:00, 8.34MB/s]


In [14]:
def obtener_datos_grafo(grafo):
    nodos = grafo.get_node_table()

    matriz_distancia = grafo.get_distance_matrix().values
    coords = nodos[['x', 'y']].values
    demandas = nodos['demand'].values

    return coords, matriz_distancia, demandas

In [15]:
benchmark_consulta = pd.read_excel("carpeta_descomprimida/problemset/problemset.xlsx")


In [16]:
def extraer_capacidades(problema):
  capacidades_disponibles = benchmark_consulta[benchmark_consulta['problem_cluster'] == problema]['vehicle_capacity'].to_list()
  return capacidades_disponibles

def resultados(problema):
  folder = "carpeta_descomprimida/problemset/in"

  g = Graph(problema, folder)

  nodos, matriz_distancia = obtener_datos_grafo(g)
  capacidades = extraer_capacidades(problema)

  coords = nodos[['x', 'y']].values
  demandas = nodos['demand'].values


In [17]:
import pandas as pd
import os

class Graph:
    def __init__(self, problem_name, folder):
        self.problem_name = problem_name
        self.folder = folder

        nodes_file = os.path.join(folder, f"{problem_name}_nodes.parquet")
        matrix_file = os.path.join(folder, f"{problem_name}_dm.parquet")


        self.nodes_df = pd.read_parquet(nodes_file)
        self.matriz_df = pd.read_parquet(matrix_file)

    def get_node_table(self):
        return self.nodes_df

    def get_distance_matrix(self):
        return self.matriz_df


In [18]:
import time
import tracemalloc

def evaluar_algoritmo_aco(evap_rate, alpha, beta, ants, lista_de_problemas, benchmark_consulta):
    resultados = []
    contador = 0

    error_general_distancia = 0
    error_cuadrado_distancia = 0

    folder = "carpeta_descomprimida/problemset/in"

    for problema in lista_de_problemas:
        g = Graph(problema, folder)

        coords, matriz_distancia, demandas = obtener_datos_grafo(g)
        capacidades = extraer_capacidades(problema)

        nodes_df = g.get_node_table()
        dist_df = g.get_distance_matrix()

        for capacidad in capacidades:
            # Iniciar medición de tiempo y memoria
            tracemalloc.start()
            tiempo_inicio = time.time()

            clusters = greedy_cluster_mejorado(coords, demandas, capacidad, matriz_distancia)

            # Paso 2: Aplicar ACOSolver en cada cluster
            rutas_totales = []
            distancia_total = 0

            for cluster in clusters:
                sub_nodes = nodes_df.iloc[cluster].reset_index(drop=True)
                sub_dm = dist_df.iloc[cluster, cluster].reset_index(drop=True).T.reset_index(drop=True)

                solver = ACOSolver(alpha=alpha, beta=beta, rho=evap_rate, ants=ants, max_iters=15)
                rutas = solver.solve(sub_nodes, sub_dm, capacidad)

                if rutas:
                    # Mapeamos índices locales -> globales
                    rutas_globales = [[cluster[i] for i in ruta] for ruta in rutas]
                    rutas_totales.extend(rutas_globales)
                    distancia_total += solver.solution_length(rutas)

            # Finalizar medición de tiempo y memoria
            tiempo_fin = time.time()
            tiempo_ejecucion = tiempo_fin - tiempo_inicio
            memoria_actual, memoria_pico = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            cantidad_vehiculos = len(rutas_totales)

            # Paso 3: Comparar con benchmark
            prueba = benchmark_consulta[
                (benchmark_consulta['problem_cluster'] == problema) &
                (benchmark_consulta['vehicle_capacity'] == capacidad)
            ][['aco best value', 'aco n vehicles']]

            if prueba.empty:
                print(f" Problema {problema} con capacidad {capacidad} no está en benchmark")
                continue

            index = int(prueba.index[0])
            aco_best_value = prueba.loc[index, 'aco best value']
            aco_n_vehicles = prueba.loc[index, 'aco n vehicles']

            # Errores

            error_distancia = abs(distancia_total - aco_best_value)
            contador += 1
            error_general_distancia += error_distancia
            error_cuadrado_distancia += error_distancia ** 2

            # Promedios
            error_promedio_distancia = error_general_distancia / contador if contador > 0 else float('inf')
            error_cuadrad_promedio_distancia = error_cuadrado_distancia / contador if contador > 0 else float('inf')

            resultados.append({
                "problema": problema,
                "capacidad": capacidad,
                "vehiculos": cantidad_vehiculos,
                "distancia": distancia_total,
                "tiempo_segundos": tiempo_ejecucion,
                "memoria_mb": memoria_pico / (1024 * 1024),
                "problemset_aco_best_value": aco_best_value,
                "problemset_aco_n_vehicles": aco_n_vehicles,
                "error_abs_distancia": error_distancia,
                "error_rel_distancia": error_distancia / aco_best_value if aco_best_value != 0 else float('inf'),
                "error_promedio_distancia": error_promedio_distancia,
                "error_cuadrad_promedio_distancia": error_cuadrad_promedio_distancia
            })

    return resultados, error_promedio_distancia, error_cuadrad_promedio_distancia

In [19]:
import os

folder = "carpeta_descomprimida/problemset/in"
for f in os.listdir(folder):
    print(f)


medium-50n-c80_120-d10_50_nodes.parquet
big-250n-c80_120-d10_50_dm.parquet
small-10n-c30_75-d15_nodes.parquet
small-10n-c30_75-d15_dm.parquet
big-250n-c80_120-d10_50_nodes.parquet
medium-50n-c150_200-d10_50_nodes.parquet
medium-50n-c150_200-d10_50_dm.parquet
big-250n-c150_300-d10_50_nodes.parquet
small-10n-c50-d10_50_dm.parquet
medium-50n-c150_200-d15_dm.parquet
small-10n-c80_120-d10_50_nodes.parquet
small-10n-c50_70-d10_50_nodes.parquet
big-250n-c150_300-d15_nodes.parquet
small-10n-c50_70-d10_50_dm.parquet
big-250n-c150_300-d10_50_dm.parquet
medium-50n-c80_120-d10_50_dm.parquet
small-10n-c80_120-d10_50_dm.parquet
small-10n-c50_60-d10_50_nodes.parquet
small-10n-c50-d10_50_nodes.parquet
medium-50n-c150_200-d15_nodes.parquet
small-10n-c50_60-d10_50_dm.parquet
big-250n-c150_300-d15_dm.parquet


In [ ]:
# Crear lista de problemas desde el benchmark
lista_de_problemas = benchmark_consulta['problem_cluster'].unique().tolist()

# Ejecutar la evaluación
resultados, error_promedio_distancia, \
error_cuadrad_promedio_distancia = evaluar_algoritmo_aco(
    0.5,     # evap_rate
    1,       # alpha
    5,       # beta
    20,      # ants
    lista_de_problemas,
    benchmark_consulta
)

# Mostrar resultados
text = f"""
Estadísticas o métricas del algoritmo creado en comparación con el problemset:

Error promedio de la distancia: {error_promedio_distancia}
Error cuadrado promedio de la distancia: {error_cuadrad_promedio_distancia}
"""

print(text)


In [ ]:
results = pd.DataFrame(resultados)
results

In [ ]:
# resultados a excell
results.to_excel("resultadosVRP.xlsx")